In [1]:
    
from cosapp.systems import System
from wrapper import JcadNotification
from jupytercad import CadDocument
from simulate import simulate, test
import time
import json, time
CYLINDER = 'Cylinder 2' 
GEAR = 'Cut 17' 
from cosapp.drivers import NonLinearSolver

In [2]:
class Geom(System):
    def setup(self, path: str):
        super().setup()
        self.add_inward('gear', CadDocument(path))
        self.add_inward('length', 8.)
        self.add_inward('radius', 2.)
        self.add_outward('mass', 0.0)
        self.add_outward('tensor_of_inertia', [[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]])
        self.add_outward('center_of_mass', [0.,0.,0.])

    def set_shape_param(self, shape, param, value):
        yobject = self.gear._get_yobject_by_name(CYLINDER)
        if yobject:
            pyobject = json.loads(yobject.to_json())
            parameters = pyobject['parameters']
            parameters[param] = value
            with self.gear.ydoc.begin_transaction() as t:
                yobject.set(t, "parameters", parameters)
                
    def compute(self):
        time.sleep(5)
        self.mass = self.gear.get_object(CYLINDER).metadata.mass
        self.tensor_of_inertia = self.gear.get_object(GEAR).metadata.matrixOfInertia
        self.center_of_mass = self.gear.get_object(GEAR).metadata.centerOfMass
        self.set_shape_param(CYLINDER, 'Height', self.length)
        self.set_shape_param(CYLINDER, 'Radius', self.radius)

In [3]:
class Simulation(System):
    def setup(self):
        self.add_inward('center_of_mass', [0.,0.,0.])
        self.add_inward('tensor_of_inertia', [[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]])
        self.add_inward('mass', 0.)
        self.add_outward('length', 0)
        self.add_outward('radius', 0)

    def compute(self):
        self.length, self.radius = simulate(self.center_of_mass, self.tensor_of_inertia, self.mass)

In [4]:
class Component(System):

    def setup(self):
        self.add_child(Geom('geom', path ='gear.jcad'))
        self.add_child(Simulation('simu'))
        self.connect(self.geom.outwards, self.simu.inwards)
        self.connect(self.geom.inwards, self.simu.outwards)

In [5]:
head = Component('head')

In [6]:
driver = head.add_driver(NonLinearSolver('solver'))

In [7]:
head.run_drivers()

In [8]:
gear = head.geom

In [9]:
gear.center_of_mass

[-4.108799189973101e-15, -0.5, -3.538001223351958e-14]

In [10]:
head.simu.length

8.0

In [11]:
obs = JcadNotification(gear.gear)
def callback():
    obj = gear.gear.get_object(CYLINDER)
    time.sleep(1)
    return test(obj)
obs.connect(callback)